In [1]:
##Actualizando...
# Parámetros de la API
# Obtener la fecha actual
import requests
from datetime import datetime, timedelta
import pandas as pd
import time

In [47]:
##Actualizando...
# Parámetros de la API
# Obtener la fecha actual
import requests
from datetime import datetime, timedelta
import pandas as pd
import time


# Parámetros de la API
base_url = "https://trading.mav-sa.com.ar/cgi-bin/wspd_cgi.sh/WService=wsbroker1/cpd-concertacion-csv_v8.r"
user_id = "avsawebservice"
password = "avsa1601"

# Obtener la fecha actual
fecha_actual = datetime.now()

# Formato de fecha para la API (DD/MM/AA)
fecha_formato_api = "%d/%m/%y"

# Crear una lista para almacenar los datos
datos = []

# Bucle para generar fechas desde el principio del año actual hasta la fecha actual
fecha_actual = fecha_actual.replace(hour=0, minute=0, second=0, microsecond=0)  # Ajustar a la medianoche
fecha = fecha_actual
while fecha <= fecha_actual:
    # Construir la fecha en formato de texto
    fecha_texto = fecha.strftime(fecha_formato_api)

    # Construir la URL de la solicitud con la fecha actual
    url = f"{base_url}?mode=ws&an={user_id}&id=avsawebservice&password={password}&fecha=14/05/24"

    # Realizar la solicitud HTTP
    requests.packages.urllib3.disable_warnings()  # Desactivar la advertencia sobre solicitud HTTPS no verificada
    response = requests.get(url, verify=False)

    # Procesar la respuesta y almacenar los datos en la lista
    datos.append([fecha_texto] + response.text.strip().split(','))

    # Avanzar al siguiente día
    fecha += timedelta(days=1)

    # Esperar 60 segundos antes de la próxima ejecución
    time.sleep(60)

# Ahora puedes trabajar directamente con la lista de datos para realizar transformaciones
# Por ejemplo, puedes convertir la lista de datos en un DataFrame de pandas y aplicar transformaciones
df = pd.DataFrame(datos)
# Realizar transformaciones adicionales en el DataFrame si es necesario
print(df)

        0                                                  1    \
0  15/05/24  Subasta; Segmento; Moneda; C-V; Tasa; Contrapa...   

                                                 2         3       4     5    \
0  89;406;13:45:43;14/05/2024;15/05/2024;1627091;...  00;67867  04;447  66;0   

                                                 6    \
0  00;983011;30-60473101-8;13:40:43;;EX;*ARP24060...   

                                       7    \
0  01;;;406;;;;;\r\n1067129;Avalado;$;V;36   

                                                 8         9       10    11   \
0  89;406;13:45:43;14/05/2024;15/05/2024;1627092;...  00;55369  45;365  23;0   

                                                 12   \
0  00;983011;30-60473101-8;13:40:43;;EX;*ARP13060...   

                                      13   \
0  7;;;406;;;;;\r\n1067180;Avalado;$;V;30   

                                                 14        15      16    17   \
0  00;523;12:00:38;14/05/2024;15/05/2024;1626691;...  9

In [48]:
import csv

fecha_hoy = datetime.now().strftime("%Y-%m-%d")

# Generar el nombre del archivo CSV con la fecha de hoy
nombre_archivo = f"datos_{fecha_hoy}.csv"

# Escribir los datos en el archivo CSV
with open(nombre_archivo, "w", newline="", encoding="utf-8") as archivo:
    escritor_csv = csv.writer(archivo)
    escritor_csv.writerows(datos)

# Leer los datos del archivo CSV y crear un DataFrame
df = pd.read_csv(nombre_archivo, delimiter=';', quotechar='"', lineterminator='\n')

# Mostrar el DataFrame
print(df)

    15/05/24,"Subasta        Segmento  Moneda  C-V    Tasa   Contraparte  \
0             1067129         Avalado       $    V  36",89           406   
1             1067129         Avalado       $    V  36",89           406   
2             1067180         Avalado       $    V  30",00           523   
3             1067180         Avalado       $    V  30",00           523   
4             1067181  No Garantizado       $    V  50",00           416   
5             1067181  No Garantizado       $    V  50",00           416   
6             1067181  No Garantizado       $    V  50",00           416   
7             1067181  No Garantizado       $    V  50",00           416   
8             1067181  No Garantizado       $    V  50",00           416   
9             1067181  No Garantizado       $    V  50",00           416   
10            1067181  No Garantizado       $    V  50",00           416   
11            1067181  No Garantizado       $    V  50",00           416   
12          

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   15/05/24,"Subasta       24 non-null     int64  
 1    Segmento               24 non-null     object 
 2    Moneda                 24 non-null     object 
 3    C-V                    24 non-null     object 
 4    Tasa                   24 non-null     object 
 5    Contraparte            24 non-null     int64  
 6    Hora Conc.             24 non-null     object 
 7    Concertación           24 non-null     object 
 8    Liquidación            24 non-null     object 
 9    Reg.Oper.              24 non-null     int64  
 10   Id.Cheque              24 non-null     int64  
 11   Banco                  24 non-null     int64  
 12   Nro.Cheque             24 non-null     int64  
 13   Acreditación           24 non-null     object 
 14   Monto                  24 non-null     obje

In [50]:
df = df.astype(str)


In [51]:
# Función para extraer la fecha y otros valores de la primera columna
def separar_fecha_valor(row):
    valores = row[0].split(',')
    fecha = valores[0].strip()
    otros_valores = ','.join(valores[2:]).strip()
    return pd.Series([fecha, otros_valores])

# Aplicar la función a cada fila del DataFrame y asignar los resultados a nuevas columnas
df[['Fecha', 'Subasta']] = df.apply(separar_fecha_valor, axis=1)

# Mostrar el DataFrame resultante
print(df)


   15/05/24,"Subasta        Segmento  Moneda  C-V    Tasa  Contraparte  \
0            1067129         Avalado       $    V  36",89          406   
1            1067129         Avalado       $    V  36",89          406   
2            1067180         Avalado       $    V  30",00          523   
3            1067180         Avalado       $    V  30",00          523   
4            1067181  No Garantizado       $    V  50",00          416   
5            1067181  No Garantizado       $    V  50",00          416   
6            1067181  No Garantizado       $    V  50",00          416   
7            1067181  No Garantizado       $    V  50",00          416   
8            1067181  No Garantizado       $    V  50",00          416   
9            1067181  No Garantizado       $    V  50",00          416   
10           1067181  No Garantizado       $    V  50",00          416   
11           1067181  No Garantizado       $    V  50",00          416   
12           1067181  No Garantizado  

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8780\2368009279.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  valores = row[0].split(',')


In [52]:
# Supongamos que tienes un DataFrame llamado df con una columna 'fecha'

# Define una función para procesar los valores de la columna 'fecha'
def procesar_fecha(valor):
    if "/" in str(valor):
        return valor
    else:
        return None  # O cualquier otro valor que desees poner en la columna 'otros_valores'

# Aplica la función a la columna 'fecha' y guarda los resultados en una nueva columna 'fecha_procesada'
df['Fecha'] = df['Fecha'].apply(procesar_fecha)

# Crea una nueva columna 'otros_valores' con los valores que no contenían '/'
df['Subasta'] = df['Fecha'].apply(lambda x: x if "/" not in str(x) else None)

In [53]:
df['Fecha'] = df['Fecha'].fillna(df[' Concertación'])


In [54]:
df

,"15/05/24,""Subasta",Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,Fecha,Subasta
0,1067129,Avalado,$,V,"36"",89",406,13:45:43,14/05/2024,15/05/2024,1627091,1616916,17,0,26/06/2024,"1666666,00","67867,04","447,66","0,00",983011,30-60473101-8,13:40:43,nan,EX,*ARP240600107,$,1,No,No,24/06/2024,FRANCES,282,No,Si,ECHEQ,CVSA,6LEN8KKQ4PO9M1Y,nan,nan,"94,""01",nan,nan,406,nan,nan,nan,nan,\r,14/05/2024,None
1,1067129,Avalado,$,V,"36"",89",406,13:45:43,14/05/2024,15/05/2024,1627092,1616917,17,0,18/06/2024,"1666666,00","55369,45","365,23","0,00",983011,30-60473101-8,13:40:43,nan,EX,*ARP130600127,$,1,No,No,13/06/2024,FRANCES,282,No,Si,ECHEQ,CVSA,LMDN388G4VK2E7R,nan,nan,"76,""7",nan,nan,406,nan,nan,nan,nan,\r,14/05/2024,None
2,1067180,Avalado,$,V,"30"",00",523,12:00:38,14/05/2024,15/05/2024,1626691,1616450,7,0,06/06/2024,"1463604,93","25995,14","210,85","0,00",390041,33-50000517-9,11:58:36,nan,EX,*GAR040600191,$,1,No,No,04/06/2024,GALICIA,109,No,Si,ECHEQ,CVSA,Q619L5VD83V27JZ,nan,nan,"44,""28",nan,nan,523,nan,nan,nan,nan,\r,14/05/2024,None
3,1067180,Avalado,$,V,"30"",00",523,12:00:38,14/05/2024,15/05/2024,1626692,1616451,7,0,07/06/2024,"1463604,93","27154,81","220,26","0,00",390041,33-50000517-9,11:58:36,nan,EX,*GAR050600227,$,1,No,No,05/06/2024,GALICIA,109,No,Si,ECHEQ,CVSA,RPQ2G8KZOX02GLY,nan,nan,"46,""25",nan,nan,523,nan,nan,nan,nan,\r,14/05/2024,None
4,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627145,1616627,11,797914,17/07/2024,"4500000,00","357503,15","1739,85","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV150700680,$,1,No,No,15/07/2024,NACION,109,Si,Si,ECHEQ,CVSA,V8794XXK6JJ9PEY,5186.0,1090021417.0,"365,""37",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,14/05/2024,None
5,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627144,1616622,11,797913,16/07/2024,"4500000,00","352272,73","1714,39","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV120700730,$,1,No,No,12/07/2024,NACION,109,Si,Si,ECHEQ,CVSA,GYKNXJJRPVP9MPR,5186.0,1090021417.0,"360,""02",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,14/05/2024,None
6,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627136,1616587,11,797917,23/07/2024,"4500000,00","388610,76","1891,24","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV190700572,$,1,No,No,19/07/2024,NACION,109,Si,Si,ECHEQ,CVSA,4MR2ZDD1VEL9DEZ,5186.0,1090021417.0,"397,""16",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,14/05/2024,None
7,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627137,1616590,11,797918,24/07/2024,"4500000,00","393750,00","1916,25","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV220700584,$,1,No,No,22/07/2024,NACION,109,Si,Si,ECHEQ,CVSA,O172VVV15712KJ3,5186.0,1090021417.0,"402,""41",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,14/05/2024,None
8,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627138,1616604,11,797907,04/07/2024,"4500000,00","288461,54","1403,85","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV020700322,$,1,No,No,02/07/2024,NACION,109,Si,Si,ECHEQ,CVSA,O172VVV15X12KJ3,5186.0,1090021417.0,"294,""81",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,14/05/2024,None
9,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627139,1616607,11,797908,05/07/2024,"4500000,00","293854,03","1430,09","0,00",310600,30-71259240-7,11:30:48,nan,RI,*

In [55]:
primera_columna_duplicada = df.iloc[:, 0]

# Crear la columna 'otros_valores'
df['Subasta'] = primera_columna_duplicada.apply(lambda x: x if '/' not in str(x) else None)

In [36]:
df

,"12/05/24,""Subasta",Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,Fecha,Subasta
0,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622840,1612563,72,0,26/08/2024,"2000000,00","222893,03","1066,26","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV220800027,$,1,No,No,22/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,LMDN38PW5EK2E7R,nan,nan,"223,""91",nan,nan,449,nan,nan,nan,nan,\r,08/05/2024,1064961
1,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622841,1612564,72,0,23/08/2024,"2000000,00","217425,28","1069,54","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV210800018,$,1,No,No,21/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,P68NK7OW1ZP9R7O,nan,nan,"224,""6",nan,nan,449,nan,nan,nan,nan,\r,08/05/2024,1064961
2,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622842,1612565,72,0,28/08/2024,"2000000,00","226519,61","1064,09","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV260800036,$,1,No,No,26/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,P68NK7OWQ5D9R7O,nan,nan,"223,""46",nan,nan,449,nan,nan,nan,nan,\r,08/05/2024,1064961
3,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622843,1612566,72,0,22/08/2024,"2000000,00","215595,21","1070,64","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV200800033,$,1,No,No,20/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,RZP90EQP1QY9EGJ,nan,nan,"224,""83",nan,nan,449,nan,nan,nan,nan,\r,08/05/2024,1064961
4,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622844,1612567,72,0,27/08/2024,"2000000,00","224708,17","1065,18","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV230800030,$,1,No,No,23/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,W612E0PYDJO2GXY,nan,nan,"223,""69",nan,nan,449,nan,nan,nan,nan,\r,08/05/2024,1064961
5,1065086,Avalado,$,V,"42"",00",523,14:02:51,08/05/2024,09/05/2024,1622864,1613280,7,0,25/07/2024,"25000000,00","2034781,30","11788,81","0,00",240045,30-70496099-5,13:59:17,nan,EX,*CVA230700001,$,1,No,No,23/07/2024,GALICIA,138,No,Si,ECHEQ,CVSA,V8794XQWP6D9PEY,nan,nan,"2475,""65",nan,nan,523,nan,nan,nan,nan,\r,08/05/2024,1065086
6,1065086,Avalado,$,V,"42"",00",523,14:02:51,08/05/2024,09/05/2024,1622865,1613281,7,0,09/08/2024,"25000000,00","2393221,68","13564,07","0,00",240045,30-70496099-5,13:59:17,nan,EX,*CVA070800003,$,1,No,No,07/08/2024,GALICIA,138,No,Si,ECHEQ,CVSA,ZEPN63Z84VPNQ0M,nan,nan,"2848,""45",nan,nan,523,nan,nan,nan,nan,\r,08/05/2024,1065086
7,1065107,Avalado,$,V,"27"",49",461,13:17:25,08/05/2024,09/05/2024,1622767,1613242,34,0,26/06/2024,"33671332,60","1174790,08","10398,89","0,00",290028,30-70496099-5,13:00:21,nan,EX,*GAR200600304,$,1,No,No,20/06/2024,PATAGONIA,304,No,Si,ECHEQ,CVSA,4MR2ZD5VXJL9DEZ,nan,nan,"2183,""77",nan,nan,114,nan,nan,nan,nan,\r,08/05/2024,1065107
8,1065107,Avalado,$,V,"27"",49",461,13:17:25,08/05/2024,09/05/2024,1622768,1613250,285,0,24/06/2024,"4387324,07","146908,89","1300,39","0,00",290028,30-70496099-5,13:00:21,nan,EX,*GAR150600478,$,1,No,No,15/06/2024,MACRO,360,No,Si,ECHEQ,CVSA,KYZ9QEP7ELP9V50,nan,nan,"273,""08",nan,nan,114,nan,nan,nan,nan,\r,08/05/2024,1065107
9,1065107,Avalado,$,V,"27"",49",461,13:17:25,08/05/2024,09/05/2024,1622769,1613251,285,0,14/06/2024,"4976152,98","131358,96","1162,75","0,00",290028,30-70496099-5,13:00:21,nan,EX,*GAR120600226,$,1,No,No,12/06/2024,MACRO,360,No,Si,ECHEQ,CVSA,LMDN38PEZK62E7R,nan,nan,"244,""18",nan,nan,114,nan,nan,nan,nan,\r,08/05/2024,1065107


In [56]:


# Eliminar las dos primeras columnas
df = df.drop(df.columns[[0]], axis=1)
df = df.drop_duplicates()

# Obtener una lista con el nombre de todas las columnas
columnas = df.columns.tolist()

# Reorganizar la lista de columnas para que "Fecha" y "Otros_Valores" estén al principio
columnas = ['Fecha', 'Subasta'] + [col for col in columnas if col not in ['Fecha', 'Subasta']]

# Reorganizar el DataFrame con las nuevas columnas
df = df[columnas]




In [38]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r
0,08/05/2024,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622840,1612563,72,0,26/08/2024,"2000000,00","222893,03","1066,26","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV220800027,$,1,No,No,22/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,LMDN38PW5EK2E7R,nan,nan,"223,""91",nan,nan,449,nan,nan,nan,nan,\r
1,08/05/2024,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622841,1612564,72,0,23/08/2024,"2000000,00","217425,28","1069,54","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV210800018,$,1,No,No,21/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,P68NK7OW1ZP9R7O,nan,nan,"224,""6",nan,nan,449,nan,nan,nan,nan,\r
2,08/05/2024,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622842,1612565,72,0,28/08/2024,"2000000,00","226519,61","1064,09","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV260800036,$,1,No,No,26/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,P68NK7OWQ5D9R7O,nan,nan,"223,""46",nan,nan,449,nan,nan,nan,nan,\r
3,08/05/2024,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622843,1612566,72,0,22/08/2024,"2000000,00","215595,21","1070,64","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV200800033,$,1,No,No,20/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,RZP90EQP1QY9EGJ,nan,nan,"224,""83",nan,nan,449,nan,nan,nan,nan,\r
4,08/05/2024,1064961,Avalado,$,V,"42"",00",449,13:47:07,08/05/2024,09/05/2024,1622844,1612567,72,0,27/08/2024,"2000000,00","224708,17","1065,18","0,00",950026,30-57612427-5,13:45:06,nan,EX,*NOV230800030,$,1,No,No,23/08/2024,SANTANDER,277,No,Si,ECHEQ,CVSA,W612E0PYDJO2GXY,nan,nan,"223,""69",nan,nan,449,nan,nan,nan,nan,\r
5,08/05/2024,1065086,Avalado,$,V,"42"",00",523,14:02:51,08/05/2024,09/05/2024,1622864,1613280,7,0,25/07/2024,"25000000,00","2034781,30","11788,81","0,00",240045,30-70496099-5,13:59:17,nan,EX,*CVA230700001,$,1,No,No,23/07/2024,GALICIA,138,No,Si,ECHEQ,CVSA,V8794XQWP6D9PEY,nan,nan,"2475,""65",nan,nan,523,nan,nan,nan,nan,\r
6,08/05/2024,1065086,Avalado,$,V,"42"",00",523,14:02:51,08/05/2024,09/05/2024,1622865,1613281,7,0,09/08/2024,"25000000,00","2393221,68","13564,07","0,00",240045,30-70496099-5,13:59:17,nan,EX,*CVA070800003,$,1,No,No,07/08/2024,GALICIA,138,No,Si,ECHEQ,CVSA,ZEPN63Z84VPNQ0M,nan,nan,"2848,""45",nan,nan,523,nan,nan,nan,nan,\r
7,08/05/2024,1065107,Avalado,$,V,"27"",49",461,13:17:25,08/05/2024,09/05/2024,1622767,1613242,34,0,26/06/2024,"33671332,60","1174790,08","10398,89","0,00",290028,30-70496099-5,13:00:21,nan,EX,*GAR200600304,$,1,No,No,20/06/2024,PATAGONIA,304,No,Si,ECHEQ,CVSA,4MR2ZD5VXJL9DEZ,nan,nan,"2183,""77",nan,nan,114,nan,nan,nan,nan,\r
8,08/05/2024,1065107,Avalado,$,V,"27"",49",461,13:17:25,08/05/2024,09/05/2024,1622768,1613250,285,0,24/06/2024,"4387324,07","146908,89","1300,39","0,00",290028,30-70496099-5,13:00:21,nan,EX,*GAR150600478,$,1,No,No,15/06/2024,MACRO,360,No,Si,ECHEQ,CVSA,KYZ9QEP7ELP9V50,nan,nan,"273,""08",nan,nan,114,nan,nan,nan,nan,\r
9,08/05/2024,1065107,Avalado,$,V,"27"",49",461,13:17:25,08/05/2024,09/05/2024,1622769,1613251,285,0,14/06/2024,"4976152,98","131358,96","1162,75","0,00",290028,30-70496099-5,13:00:21,nan,EX,*GAR120600226,$,1,No,No,12/06/2024,MACRO,360,No,Si,ECHEQ,CVSA,LMDN38PEZK62E7R,nan,nan,"244,""18",nan,nan,114,nan,nan,nan,nan,\r


In [57]:
from datetime import datetime

# Convertir 'Fecha' a tipo datetime con el formato adecuado
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')

# Convertir 'Vto.' a tipo datetime con el formato adecuado
df['Vto.'] = pd.to_datetime(df['Vto.'], format='%d/%m/%Y')
df['dias_entre_fechas'] = (df['Vto.'] - df['Fecha']).dt.days

In [58]:
# Eliminar las comas de las cadenas y convertirlas a números de punto flotante
df[' Monto'] = df[' Monto'].str.split(',').str[0]
# Define una función para asignar la categoría en función de las palabras clave
def asignar_categoria(texto):
    if 'ARP' in texto:
        return 'ARPY'
    elif 'GAR' in texto:
        return 'GAR'
    elif 'MAV' in texto:
        return 'MAV'
    elif 'ACE' in texto:
        return 'ACEN'
    elif 'FDE' in texto:
        return 'FIDEM'
    elif 'NOV' in texto:
        return 'NOV'
    elif 'CRE' in texto:
        return 'CREC'
    elif 'POT' in texto:
        return 'POT'
    elif 'ACI' in texto:
        return 'ACIN'
    elif 'POT' in texto:
        return 'POT'
    elif 'BIN' in texto:
        return 'BIND'
    elif '#UGA' in texto:
        return 'GAR'
    elif '#UFA' in texto:
        return 'AVFE'
    elif '#UAR' in texto:
        return 'ARP'
    elif '#UAM' in texto:
        return 'AMER' 
    elif '#UAC' in texto:
        return 'ACIN'
    elif 'FED' in texto:
        return 'FEDE'
    elif 'FINT' in texto:
        return 'FINT'
    elif 'FTR' in texto:
        return 'AVFE'
    
    
    # AgregaMOS más condiciones según sea necesario
    else:
        return 'otra_categoria'

# Aplica la función a la columna 'texto' para crear la nueva columna 'SGR'
df['SGR'] = df[' Cod.Ch.'].apply(asignar_categoria)

# Muestra el DataFrame con la nueva columna
print(df)

        Fecha  Subasta        Segmento  Moneda  C-V    Tasa  Contraparte  \
0  2024-05-14  1067129         Avalado       $    V  36",89          406   
1  2024-05-14  1067129         Avalado       $    V  36",89          406   
2  2024-05-14  1067180         Avalado       $    V  30",00          523   
3  2024-05-14  1067180         Avalado       $    V  30",00          523   
4  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
5  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
6  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
7  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
8  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
9  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
10 2024-05-14  1067181  No Garantizado       $    V  50",00          416   
11 2024-05-14  1067181  No Garantizado       $    V  50",00          416   
12 2024-05-1

In [59]:
# Configurar la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ahora puedes imprimir el DataFrame y ver todas las columnas
print(df)

        Fecha  Subasta        Segmento  Moneda  C-V    Tasa  Contraparte  \
0  2024-05-14  1067129         Avalado       $    V  36",89          406   
1  2024-05-14  1067129         Avalado       $    V  36",89          406   
2  2024-05-14  1067180         Avalado       $    V  30",00          523   
3  2024-05-14  1067180         Avalado       $    V  30",00          523   
4  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
5  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
6  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
7  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
8  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
9  2024-05-14  1067181  No Garantizado       $    V  50",00          416   
10 2024-05-14  1067181  No Garantizado       $    V  50",00          416   
11 2024-05-14  1067181  No Garantizado       $    V  50",00          416   
12 2024-05-1

In [60]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-05-14,1067129,Avalado,$,V,"36"",89",406,13:45:43,14/05/2024,15/05/2024,1627091,1616916,17,0,26/06/2024,1666666,"67867,04","447,66","0,00",983011,30-60473101-8,13:40:43,nan,EX,*ARP240600107,$,1,No,No,2024-06-24,FRANCES,282,No,Si,ECHEQ,CVSA,6LEN8KKQ4PO9M1Y,nan,nan,"94,""01",nan,nan,406,nan,nan,nan,nan,\r,41,ARPY
1,2024-05-14,1067129,Avalado,$,V,"36"",89",406,13:45:43,14/05/2024,15/05/2024,1627092,1616917,17,0,18/06/2024,1666666,"55369,45","365,23","0,00",983011,30-60473101-8,13:40:43,nan,EX,*ARP130600127,$,1,No,No,2024-06-13,FRANCES,282,No,Si,ECHEQ,CVSA,LMDN388G4VK2E7R,nan,nan,"76,""7",nan,nan,406,nan,nan,nan,nan,\r,30,ARPY
2,2024-05-14,1067180,Avalado,$,V,"30"",00",523,12:00:38,14/05/2024,15/05/2024,1626691,1616450,7,0,06/06/2024,1463604,"25995,14","210,85","0,00",390041,33-50000517-9,11:58:36,nan,EX,*GAR040600191,$,1,No,No,2024-06-04,GALICIA,109,No,Si,ECHEQ,CVSA,Q619L5VD83V27JZ,nan,nan,"44,""28",nan,nan,523,nan,nan,nan,nan,\r,21,GAR
3,2024-05-14,1067180,Avalado,$,V,"30"",00",523,12:00:38,14/05/2024,15/05/2024,1626692,1616451,7,0,07/06/2024,1463604,"27154,81","220,26","0,00",390041,33-50000517-9,11:58:36,nan,EX,*GAR050600227,$,1,No,No,2024-06-05,GALICIA,109,No,Si,ECHEQ,CVSA,RPQ2G8KZOX02GLY,nan,nan,"46,""25",nan,nan,523,nan,nan,nan,nan,\r,22,GAR
4,2024-05-14,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627145,1616627,11,797914,17/07/2024,4500000,"357503,15","1739,85","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV150700680,$,1,No,No,2024-07-15,NACION,109,Si,Si,ECHEQ,CVSA,V8794XXK6JJ9PEY,5186.0,1090021417.0,"365,""37",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,62,MAV
5,2024-05-14,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627144,1616622,11,797913,16/07/2024,4500000,"352272,73","1714,39","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV120700730,$,1,No,No,2024-07-12,NACION,109,Si,Si,ECHEQ,CVSA,GYKNXJJRPVP9MPR,5186.0,1090021417.0,"360,""02",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,59,MAV
6,2024-05-14,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627136,1616587,11,797917,23/07/2024,4500000,"388610,76","1891,24","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV190700572,$,1,No,No,2024-07-19,NACION,109,Si,Si,ECHEQ,CVSA,4MR2ZDD1VEL9DEZ,5186.0,1090021417.0,"397,""16",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,66,MAV
7,2024-05-14,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627137,1616590,11,797918,24/07/2024,4500000,"393750,00","1916,25","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV220700584,$,1,No,No,2024-07-22,NACION,109,Si,Si,ECHEQ,CVSA,O172VVV15712KJ3,5186.0,1090021417.0,"402,""41",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,69,MAV
8,2024-05-14,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627138,1616604,11,797907,04/07/2024,4500000,"288461,54","1403,85","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV020700322,$,1,No,No,2024-07-02,NACION,109,Si,Si,ECHEQ,CVSA,O172VVV15X12KJ3,5186.0,1090021417.0,"294,""81",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,49,MAV
9,2024-05-14,1067181,No Garantizado,$,V,"50"",00",416,13:55:41,14/05/2024,15/05/2024,1627139,1616607,11,797908,05/07/2024,4500000,"293854,03","1430,09","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV030700433,$,1,N

In [61]:
df[' Tasa'] = df[' Tasa'].str.replace('"', '')

# Reemplazar la coma por un punto en los valores de la columna "tasa"
df[' Tasa'] = df[' Tasa'].str.replace(',', '.')

# Convertir la columna "tasa" a tipo float
df[' Tasa'] = df[' Tasa'].astype(float)

# Redondear los valores en la columna "tasa" al entero más cercano
df[' Tasa'] = df[' Tasa'].round()

In [62]:
columnas_a_ignorar = ['Fecha', 'Vto.',' Hora Conc.',' Cod.Ch.','SGR',' Moneda Liquidacion','Nombre Banco','Razón Librador/Deudor\r','Cond. Pyme', ' Banco',' C-V',' Hora Ord.',' Hora Conc.', ' Concertación', ' Liquidación', ' Reg.Oper.',' Cond.',' Banco',' Acreditación']



for columna in df.columns:
    if columna not in columnas_a_ignorar:
        # Verifica si los valores de la columna son numéricos
        if pd.api.types.is_numeric_dtype(df[columna]):
            # Convierte la columna a tipo de datos Int64
            df[columna] = df[columna].astype('Int64')

# Muestra el DataFrame con las columnas convertidas
print(df)


df[' Monto'] = df[' Monto'].astype(int)


        Fecha  Subasta        Segmento  Moneda  C-V   Tasa  Contraparte  \
0  2024-05-14  1067129         Avalado       $    V     37          406   
1  2024-05-14  1067129         Avalado       $    V     37          406   
2  2024-05-14  1067180         Avalado       $    V     30          523   
3  2024-05-14  1067180         Avalado       $    V     30          523   
4  2024-05-14  1067181  No Garantizado       $    V     50          416   
5  2024-05-14  1067181  No Garantizado       $    V     50          416   
6  2024-05-14  1067181  No Garantizado       $    V     50          416   
7  2024-05-14  1067181  No Garantizado       $    V     50          416   
8  2024-05-14  1067181  No Garantizado       $    V     50          416   
9  2024-05-14  1067181  No Garantizado       $    V     50          416   
10 2024-05-14  1067181  No Garantizado       $    V     50          416   
11 2024-05-14  1067181  No Garantizado       $    V     50          416   
12 2024-05-14  1067181  N

In [63]:
df = df.drop_duplicates("Subasta")


In [64]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-05-14,1067129,Avalado,$,V,37,406,13:45:43,14/05/2024,15/05/2024,1627091,1616916,17,0,26/06/2024,1666666,"67867,04","447,66","0,00",983011,30-60473101-8,13:40:43,nan,EX,*ARP240600107,$,1,No,No,2024-06-24,FRANCES,282,No,Si,ECHEQ,CVSA,6LEN8KKQ4PO9M1Y,nan,nan,"94,""01",nan,nan,406,nan,nan,nan,nan,\r,41,ARPY
2,2024-05-14,1067180,Avalado,$,V,30,523,12:00:38,14/05/2024,15/05/2024,1626691,1616450,7,0,06/06/2024,1463604,"25995,14","210,85","0,00",390041,33-50000517-9,11:58:36,nan,EX,*GAR040600191,$,1,No,No,2024-06-04,GALICIA,109,No,Si,ECHEQ,CVSA,Q619L5VD83V27JZ,nan,nan,"44,""28",nan,nan,523,nan,nan,nan,nan,\r,21,GAR
4,2024-05-14,1067181,No Garantizado,$,V,50,416,13:55:41,14/05/2024,15/05/2024,1627145,1616627,11,797914,17/07/2024,4500000,"357503,15","1739,85","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV150700680,$,1,No,No,2024-07-15,NACION,109,Si,Si,ECHEQ,CVSA,V8794XXK6JJ9PEY,5186.0,1090021417.0,"365,""37",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,62,MAV
16,2024-05-14,1067244,No Garantizado,$,V,48,416,12:49:10,14/05/2024,15/05/2024,1626936,1617304,285,92755339,12/06/2024,25775433,"915394,34","4640,54","0,00",340412,30-71259240-7,12:47:08,nan,RI,*MAV100600722,$,1,No,No,2024-06-10,MACRO,300,Si,Si,ECHEQ,CVSA,JWV9JGGZ6039QMD,5000.0,17299.0,"974,""51",33-70796584-9,nan,416,AO,nan,nan,Certificado MiPyME,HELACOR SA\r,27,MAV
17,2024-05-14,1067364,Avalado,$,V,37,406,13:42:56,14/05/2024,15/05/2024,1627082,1617703,72,0,17/07/2024,30000000,"1800880,74","11843,63","0,00",983029,30-60473101-8,13:40:54,nan,EX,*ARP140700113,$,1,No,No,2024-07-14,SANTANDER,520,No,Si,ECHEQ,CVSA,Q619L55585327JZ,nan,nan,"2487,""16",nan,nan,406,nan,nan,nan,nan,\r,61,ARPY
18,2024-05-14,1067428,Avalado,$,V,36,406,14:32:42,14/05/2024,15/05/2024,1627245,1617825,11,0,05/09/2024,3000000,"300828,24","1619,50","0,00",310600,30-50001091-2,14:30:41,nan,EX,*ACI030900054,$,1,No,No,2024-09-03,NACION,109,No,Si,ECHEQ,CVSA,EZ8NPRRZ1QK24KV,nan,nan,"340,""1",nan,nan,406,nan,nan,nan,nan,\r,112,ACIN
21,2024-05-14,1067548,Avalado,$,V,35,15,15:22:06,14/05/2024,15/05/2024,1627428,1618009,7,0,03/09/2024,10000000,"961990,84","5422,81","0,00",983014,30-53467243-4,15:12:20,nan,RI,*GAR300800298,$,1,No,No,2024-08-30,GALICIA,111,No,Si,ECHEQ,CVSA,4OGNYLLVO6M20L6,nan,nan,"1138,""79",nan,nan,15,nan,nan,nan,nan,\r,108,GAR
23,2024-05-14,1067551,No Garantizado,$,V,55,416,15:43:53,14/05/2024,15/05/2024,1627526,1618049,72,497,14/08/2024,10000000,"1205878,81","5276,47","0,00",230010,30-71259240-7,15:41:51,nan,RI,*MAV120800539,$,1,No,No,2024-08-12,SANTANDER,246,Si,Si,ECHEQ,CVSA,4OGNYLL77V420L6,5220.0,18586.0,"1108,06",30-71208466-5,nan,416,AO,nan,nan,Certificado MiPyME,INTERCAMPOS S.R.L.\r,90,MAV


In [65]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-05-14,1067129,Avalado,$,V,37,406,13:45:43,14/05/2024,15/05/2024,1627091,1616916,17,0,26/06/2024,1666666,"67867,04","447,66","0,00",983011,30-60473101-8,13:40:43,nan,EX,*ARP240600107,$,1,No,No,2024-06-24,FRANCES,282,No,Si,ECHEQ,CVSA,6LEN8KKQ4PO9M1Y,nan,nan,"94,""01",nan,nan,406,nan,nan,nan,nan,\r,41,ARPY
2,2024-05-14,1067180,Avalado,$,V,30,523,12:00:38,14/05/2024,15/05/2024,1626691,1616450,7,0,06/06/2024,1463604,"25995,14","210,85","0,00",390041,33-50000517-9,11:58:36,nan,EX,*GAR040600191,$,1,No,No,2024-06-04,GALICIA,109,No,Si,ECHEQ,CVSA,Q619L5VD83V27JZ,nan,nan,"44,""28",nan,nan,523,nan,nan,nan,nan,\r,21,GAR
4,2024-05-14,1067181,No Garantizado,$,V,50,416,13:55:41,14/05/2024,15/05/2024,1627145,1616627,11,797914,17/07/2024,4500000,"357503,15","1739,85","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV150700680,$,1,No,No,2024-07-15,NACION,109,Si,Si,ECHEQ,CVSA,V8794XXK6JJ9PEY,5186.0,1090021417.0,"365,""37",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,62,MAV
16,2024-05-14,1067244,No Garantizado,$,V,48,416,12:49:10,14/05/2024,15/05/2024,1626936,1617304,285,92755339,12/06/2024,25775433,"915394,34","4640,54","0,00",340412,30-71259240-7,12:47:08,nan,RI,*MAV100600722,$,1,No,No,2024-06-10,MACRO,300,Si,Si,ECHEQ,CVSA,JWV9JGGZ6039QMD,5000.0,17299.0,"974,""51",33-70796584-9,nan,416,AO,nan,nan,Certificado MiPyME,HELACOR SA\r,27,MAV
17,2024-05-14,1067364,Avalado,$,V,37,406,13:42:56,14/05/2024,15/05/2024,1627082,1617703,72,0,17/07/2024,30000000,"1800880,74","11843,63","0,00",983029,30-60473101-8,13:40:54,nan,EX,*ARP140700113,$,1,No,No,2024-07-14,SANTANDER,520,No,Si,ECHEQ,CVSA,Q619L55585327JZ,nan,nan,"2487,""16",nan,nan,406,nan,nan,nan,nan,\r,61,ARPY
18,2024-05-14,1067428,Avalado,$,V,36,406,14:32:42,14/05/2024,15/05/2024,1627245,1617825,11,0,05/09/2024,3000000,"300828,24","1619,50","0,00",310600,30-50001091-2,14:30:41,nan,EX,*ACI030900054,$,1,No,No,2024-09-03,NACION,109,No,Si,ECHEQ,CVSA,EZ8NPRRZ1QK24KV,nan,nan,"340,""1",nan,nan,406,nan,nan,nan,nan,\r,112,ACIN
21,2024-05-14,1067548,Avalado,$,V,35,15,15:22:06,14/05/2024,15/05/2024,1627428,1618009,7,0,03/09/2024,10000000,"961990,84","5422,81","0,00",983014,30-53467243-4,15:12:20,nan,RI,*GAR300800298,$,1,No,No,2024-08-30,GALICIA,111,No,Si,ECHEQ,CVSA,4OGNYLLVO6M20L6,nan,nan,"1138,""79",nan,nan,15,nan,nan,nan,nan,\r,108,GAR
23,2024-05-14,1067551,No Garantizado,$,V,55,416,15:43:53,14/05/2024,15/05/2024,1627526,1618049,72,497,14/08/2024,10000000,"1205878,81","5276,47","0,00",230010,30-71259240-7,15:41:51,nan,RI,*MAV120800539,$,1,No,No,2024-08-12,SANTANDER,246,Si,Si,ECHEQ,CVSA,4OGNYLL77V420L6,5220.0,18586.0,"1108,06",30-71208466-5,nan,416,AO,nan,nan,Certificado MiPyME,INTERCAMPOS S.R.L.\r,90,MAV


In [66]:
def asignar_periodo(dias_entre_fechas):
    if dias_entre_fechas < 60:
        return '0 a 60'
    elif 60 <= dias_entre_fechas < 90:
        return '60 a 90'
    elif 90 <= dias_entre_fechas < 120:
        return '90 a 120'
    elif 120 <= dias_entre_fechas < 180:
        return '120 a 180'
    elif 180 <= dias_entre_fechas < 210:
        return '180 a 210'
    elif 210 <= dias_entre_fechas < 240:
        return '210 a 240'
    elif 240 <= dias_entre_fechas < 300:
        return '240 a 300'
    else:
        return 'Más de 300'

# Aplicaa mosla función a la columna 'dias_entre_fechas' para crear la nueva columna 'periodo'
df['Periodo'] = df['dias_entre_fechas'].apply(asignar_periodo)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8780\929678408.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Periodo'] = df['dias_entre_fechas'].apply(asignar_periodo)


In [67]:
## Reordenar las columnas
# Para obtener el listado de columnas
columnas = df.columns
print("Listado de columnas:")
print(columnas)



Listado de columnas:
Index(['Fecha', 'Subasta', ' Segmento', ' Moneda', ' C-V', ' Tasa',
       ' Contraparte', ' Hora Conc.', ' Concertación', ' Liquidación',
       ' Reg.Oper.', ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación',
       ' Monto', ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente',
       ' CUIT Comprador', ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.',
       ' Moneda Liquidacion', ' Tipo de Cambio', ' Sin Recurso',
       ' No a la Orden', 'Vto.', 'Nombre Banco', 'N. Sucursal', 'Pyme',
       'Primera.Neg', 'Tipo Instrumento', 'Custodio/Registro', 'Echeqid',
       'Plaza Cheque', 'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor',
       'CUIT PyME', 'Liquidador Compra', 'Caracter', 'CUIT Benef.',
       'Razón Benef.', 'Cond. Pyme', 'Razón Librador/Deudor\r',
       'dias_entre_fechas', 'SGR', 'Periodo'],
      dtype='object')


In [68]:
# Para reordenar las columnas, puedes proporcionar un nuevo orden de columnas
# Por ejemplo, si deseas reorganizar las columnas en un nuevo orden específico
nuevo_orden = ['Fecha', 'Subasta',' Moneda' ,' C-V', ' Tasa',
       ' Contraparte', ' Hora Conc.', ' Concertación', ' Liquidación',
       ' Reg.Oper.', ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación',
       ' Monto', ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente',
       ' CUIT Comprador', ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.',
       ' Moneda Liquidacion', ' Tipo de Cambio', ' Sin Recurso',
       ' No a la Orden', 'Vto.', 'Nombre Banco', 'N. Sucursal', 'Pyme',
       'Primera.Neg', 'Tipo Instrumento', 'Custodio/Registro', 'Echeqid',
       'Plaza Cheque', 'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor',
       'CUIT PyME', 'Liquidador Compra', 'Caracter', 'CUIT Benef.',
       'Razón Benef.', 'Cond. Pyme', 'Razón Librador/Deudor\r',
       'dias_entre_fechas', 'SGR',' Segmento','Periodo']  # Define el nuevo orden de las columnas

# Reordena el DataFrame según el nuevo orden de columnas
df = df[nuevo_orden]

In [69]:
df.rename(columns={' Segmento': 'Segmento'}, inplace=True)
df

,Fecha,Subasta,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR,Segmento,Periodo
0,2024-05-14,1067129,$,V,37,406,13:45:43,14/05/2024,15/05/2024,1627091,1616916,17,0,26/06/2024,1666666,"67867,04","447,66","0,00",983011,30-60473101-8,13:40:43,nan,EX,*ARP240600107,$,1,No,No,2024-06-24,FRANCES,282,No,Si,ECHEQ,CVSA,6LEN8KKQ4PO9M1Y,nan,nan,"94,""01",nan,nan,406,nan,nan,nan,nan,\r,41,ARPY,Avalado,0 a 60
2,2024-05-14,1067180,$,V,30,523,12:00:38,14/05/2024,15/05/2024,1626691,1616450,7,0,06/06/2024,1463604,"25995,14","210,85","0,00",390041,33-50000517-9,11:58:36,nan,EX,*GAR040600191,$,1,No,No,2024-06-04,GALICIA,109,No,Si,ECHEQ,CVSA,Q619L5VD83V27JZ,nan,nan,"44,""28",nan,nan,523,nan,nan,nan,nan,\r,21,GAR,Avalado,0 a 60
4,2024-05-14,1067181,$,V,50,416,13:55:41,14/05/2024,15/05/2024,1627145,1616627,11,797914,17/07/2024,4500000,"357503,15","1739,85","0,00",310600,30-71259240-7,11:30:48,nan,RI,*MAV150700680,$,1,No,No,2024-07-15,NACION,109,Si,Si,ECHEQ,CVSA,V8794XXK6JJ9PEY,5186.0,1090021417.0,"365,""37",30-70862775-1,nan,416,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,62,MAV,No Garantizado,60 a 90
16,2024-05-14,1067244,$,V,48,416,12:49:10,14/05/2024,15/05/2024,1626936,1617304,285,92755339,12/06/2024,25775433,"915394,34","4640,54","0,00",340412,30-71259240-7,12:47:08,nan,RI,*MAV100600722,$,1,No,No,2024-06-10,MACRO,300,Si,Si,ECHEQ,CVSA,JWV9JGGZ6039QMD,5000.0,17299.0,"974,""51",33-70796584-9,nan,416,AO,nan,nan,Certificado MiPyME,HELACOR SA\r,27,MAV,No Garantizado,0 a 60
17,2024-05-14,1067364,$,V,37,406,13:42:56,14/05/2024,15/05/2024,1627082,1617703,72,0,17/07/2024,30000000,"1800880,74","11843,63","0,00",983029,30-60473101-8,13:40:54,nan,EX,*ARP140700113,$,1,No,No,2024-07-14,SANTANDER,520,No,Si,ECHEQ,CVSA,Q619L55585327JZ,nan,nan,"2487,""16",nan,nan,406,nan,nan,nan,nan,\r,61,ARPY,Avalado,60 a 90
18,2024-05-14,1067428,$,V,36,406,14:32:42,14/05/2024,15/05/2024,1627245,1617825,11,0,05/09/2024,3000000,"300828,24","1619,50","0,00",310600,30-50001091-2,14:30:41,nan,EX,*ACI030900054,$,1,No,No,2024-09-03,NACION,109,No,Si,ECHEQ,CVSA,EZ8NPRRZ1QK24KV,nan,nan,"340,""1",nan,nan,406,nan,nan,nan,nan,\r,112,ACIN,Avalado,90 a 120
21,2024-05-14,1067548,$,V,35,15,15:22:06,14/05/2024,15/05/2024,1627428,1618009,7,0,03/09/2024,10000000,"961990,84","5422,81","0,00",983014,30-53467243-4,15:12:20,nan,RI,*GAR300800298,$,1,No,No,2024-08-30,GALICIA,111,No,Si,ECHEQ,CVSA,4OGNYLLVO6M20L6,nan,nan,"1138,""79",nan,nan,15,nan,nan,nan,nan,\r,108,GAR,Avalado,90 a 120
23,2024-05-14,1067551,$,V,55,416,15:43:53,14/05/2024,15/05/2024,1627526,1618049,72,497,14/08/2024,10000000,"1205878,81","5276,47","0,00",230010,30-71259240-7,15:41:51,nan,RI,*MAV120800539,$,1,No,No,2024-08-12,SANTANDER,246,Si,Si,ECHEQ,CVSA,4OGNYLL77V420L6,5220.0,18586.0,"1108,06",30-71208466-5,nan,416,AO,nan,nan,Certificado MiPyME,INTERCAMPOS S.R.L.\r,90,MAV,No Garantizado,90 a 120


In [21]:
df.columns

Index(['Fecha', 'Subasta', ' Moneda', ' C-V', ' Tasa', ' Contraparte',
       ' Hora Conc.', ' Concertación', ' Liquidación', ' Reg.Oper.',
       ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación', ' Monto',
       ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente', ' CUIT Comprador',
       ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.', ' Moneda Liquidacion',
       ' Tipo de Cambio', ' Sin Recurso', ' No a la Orden', 'Vto.',
       'Nombre Banco', 'N. Sucursal', 'Pyme', 'Primera.Neg',
       'Tipo Instrumento', 'Custodio/Registro', 'Echeqid', 'Plaza Cheque',
       'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor', 'CUIT PyME',
       'Liquidador Compra', 'Caracter', 'CUIT Benef.', 'Razón Benef.',
       'Cond. Pyme', 'Razón Librador/Deudor\r', 'dias_entre_fechas', 'SGR',
       'Segmento', 'Periodo'],
      dtype='object')

In [70]:
from sqlalchemy import create_engine

# Especificamos los detalles de la conexión a la base de datos PostgreSQL
usuario = 'postgres'
contraseña = 'Camila1995.'
host = 'localhost'
puerto = '5432'
base_de_datos = 'mav2024'

# Creamos la URL de conexión a la base de datos PostgreSQL
url_de_conexion = f'postgresql://{usuario}:{contraseña}@{host}:{puerto}/{base_de_datos}'

# Creamos una instancia del motor (engine) de SQLAlchemy
engine = create_engine(url_de_conexion)

# Guardamos el nuevo DataFrame en la tabla existente en la base de datos
nombre_tabla = 'mav2024'  # Especifica el nombre de la tabla donde deseas agregar los datos
df.to_sql(nombre_tabla, engine, if_exists='append', index=False)

8